# RAIS Centro-Oeste (Análise)

Análise dos arquivos da RAIS conforme separados em diferentes arquivos parquet particionados por ano.

In [1]:
import pyarrow.dataset as pds
import pyarrow.parquet as pq
import pyarrow as pa
import s3fs
import pandas as pd

In [2]:
ACCESS_KEY = ''
SECRET_KEY = ''
SOURCE_FOLDER = 'intermediate/DIEPS/bases/rais/2024'
MINIO_HOST = 'http://minio:9000'
#PANDAS_CHUNK_SIZE = 200000 # 200.000 linhas por chunk
#PANDAS_NROWS = 1000000
#PANDAS_OFFSET = 6000000
ANO = 2024

#TARGET_FILE = f'intermediate/DIEPS/bases/rais/{ANO}/rais-centro-oeste-{PANDAS_OFFSET}-{(PANDAS_OFFSET + PANDAS_NROWS) - 1}.parquet'

# Metadados
RESPONSAVEL_NOME = 'matheus.fernandes'
RESPONSAVEL_EMAIL = 'matheus.fernandes@ipe.df.gov.br'
RESPONSAVEL_SETOR = 'UCTIS/COGEI/GEINO'
PROCESSED = 'False'

In [3]:
minio = s3fs.S3FileSystem(key=ACCESS_KEY, # change default key 
                        secret=SECRET_KEY, # change default secret
                        use_ssl=False,
                        client_kwargs={
                            'endpoint_url': MINIO_HOST} # change ip
                        )

# Verifiacação da conexão 
objects = minio.head(SOURCE_FOLDER)
objects

## Carregamento em dataframe

Os arquivos parquet listados acima serão carregados em um único dataframe.


In [11]:
from io import BytesIO
fs = s3fs.core.S3FileSystem()

part = pds.partitioning(
    pa.schema([
        ('year', pa.int16())
    ])
)
dataset = pq.ParquetDataset(
    SOURCE_FOLDER,
    filesystem=minio,
    partitioning=part
)


# dt = dataset.to_table()
# dt
dataset.files


In [17]:
cols = [field.name for field in dataset.schema]
cols

In [38]:
#df = dataset.read(columns=['Bairros SP', 'Causa Afastamento 1']).to_pandas()
df = dataset.read_pandas(
    columns=[
        'Causa Afastamento 1',
        'Causa Afastamento 2',
        'Causa Afastamento 3',
        'Idade',
        'Faixa Etária'
    ]).to_pandas()

In [39]:
df

In [45]:
# Remove todos que tem 99
df = df[df['Causa Afastamento 1'] != 99]

# Analise

Verifica correlação entre causa de afastamento e faixa etária

In [46]:
df['Idade'].unique()

# Grafico

Apresenta a principal causa de afastamento por idade em um scaterplot

In [48]:
import matplotlib.pyplot as plt

df.groupby(['Causa Afastamento 1', 'Faixa Etária'])['Faixa Etária'].count().unstack('Causa Afastamento 1').plot.bar(ax=plt.gca())

#.plot(kind='scatter', y='Causa Afastamento 1', x='Idade')